<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_3_digital_marketer_rag_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🎯 Day 3 — Digital Marketer: Контент-ассистент по бренд-документу

Этот ноутбук поможет вам создать RAG-бота, который отвечает на вопросы по маркетинговому PDF-документу — брендбуку.

Мы используем:
- LlamaIndex
- VectorStoreIndex
- QueryEngine с цитированием источников

📄 Пример PDF: [Брендбук «Кит и Кот»](https://kitkot.ru/Downloadsbranbooks/Brandbook_Sveda.pdf)


In [10]:
!pip install llama-index llama-index-vector-stores-faiss PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 55.3 MB/s eta 0:00:00


🔍 Разница между SimpleDirectoryReader и PyMuPDFReader
✅ 1. SimpleDirectoryReader("data").load_data()

Это высокоуровневый загрузчик:
Проходит по всем файлам в указанной папке.
Автоматически определяет типы файлов (.txt, .pdf, .docx, .md, и т.д.).
Использует встроенные ридеры, в т.ч. PyMuPDFReader для PDF.

Плюсы:

Очень просто.
Удобно для загрузки сразу нескольких документов.
Работает "из коробки".

Минусы:

Меньше контроля над обработкой.
Иногда может проигнорировать нестандартные PDF или выдать warning.

✅ 2. PyMuPDFReader().load(file_path=...)

Это низкоуровневый, явный способ:
Ты напрямую говоришь: «используй этот ридер для этого файла».
Позволяет точнее контролировать поведение (например, разбивку по страницам, кастомные опции).

Плюсы:

Лучше для тонкой настройки.
Можно выбрать конкретный файл (а не все в папке).
Явно указываешь ридер.

Минусы:

Нужно писать чуть больше кода.
Не пройдёт по папке автоматически.

🧪 Когда что использовать?

Сценарий	                              Рекомендуемый способ
Быстро загрузить все документы в папке	SimpleDirectoryReader
Загрузить конкретный PDF с контролем	  PyMuPDFReader
Загрузка PDF с кастомной логикой	      PyMuPDFReader + параметры
Разные типы файлов в одной папке	      SimpleDirectoryReader


Если в Colab у тебя один PDF — тогда лучше прямо PyMuPDFReader.

Если будет много документов — можно оставить SimpleDirectoryReader.

In [12]:
from llama_index.core import VectorStoreIndex # Import VectorStoreIndex from llama_index.core
from llama_index.readers.file import PyMuPDFReader
from llama_index.core import SimpleDirectoryReader # Import SimpleDirectoryReader from llama_index.core

import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

# 1-й вариант - Загрузка PDF-файла
reader = PyMuPDFReader()
docs = reader.load(file_path="/content/data/Brandbook_Sveda.pdf")
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine()


Вставь OpenAI API ключ: ··········


In [13]:
'''
# 2-й вариант - Загрузка и построение индекса
docs = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine()
'''


'\n# 2-й вариант - Загрузка и построение индекса\ndocs = SimpleDirectoryReader("data").load_data()\nindex = VectorStoreIndex.from_documents(docs)\nquery_engine = index.as_query_engine()\n'

In [20]:
def ask(query):
    response = query_engine.query(query)
    print("🔹 Ответ:")
    print(response.response)
    print("\n🔸 Источники:")
    for i, node in enumerate(response.source_nodes):
        print(f"\nИсточник {i+1}:")
        print(node.get_text().strip())


In [21]:
# 🔍 Пример запроса
ask("Какие элементы офрмления интерьера?")


🔹 Ответ:
Стойка кассира-администратора, элементы оборудования (витрина), свободные поверхности стен, навигационные элементы, дизайн ценников, дополнительная коммуникация с покупателем, интерьерный текстиль.

🔸 Источники:

Источник 1:
4  /  ЭЛЕМЕНТЫ ОФОРМЛЕНИЯ ИНТЕРЬЕРА
4.1 Элементы 
оформления интерьера
Руководство даёт правила и рекомендации по 
оформлению элементов интерьера с точки зрения 
брендинга, а не по конструктивным и материаль-
ным решениям.
В руководстве рассмотрено оформление 
следующих элементов интерьера:
1. Стойка кассира-администратора
2. Элементы оборудования (витрина)
3. Свободные поверхности стен
4. Навигационные элементы
5. Дизайн ценников
6. Дополнительная коммуникация с покупателем
6. Интерьерный текстиль
Брендбук 
49

Источник 2:
4  /  ЭЛЕМЕНТЫ ОФОРМЛЕНИЯ ИНТЕРЬЕРА
4.2 Цветовое решение
Цвета
Текстуры
Фирменные цвета и стилеобразующие элементы в 
интерьере являются акцентными и используются 
дозировано. В противном случае, пространство 
может потерять воздушность


# ✅ Чеклист
# [ ] Загрузил PDF в папку `data/`
# [ ] Запустил индексацию
# [ ] Сделал 2+ запроса и получил ответы с цитатами
# [ ] Сохранил заметки

# 💬 Примеры вопросов:
# - Какие цвета использовать на лендинге?
# - Можем ли мы использовать emoji в Instagram?
# - Где упоминается логотип?
